<a href="https://colab.research.google.com/github/RashmikaD2001/Evolutionary-Computing-Assignment/blob/main/EC_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [ ]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt

SEED = 70
np.random.seed(SEED)
random.seed(SEED)

**Fitness Function**

In [ ]:
def fitness(chromosome):
    """
    Evaluates the fitness of a chromosome for the video storing problem (Slide 14).
    Fitness = sum(w * f(x)), where w = 1 if sum(g(x)) <= 4500, else w = 0.5 for each selected file.

    Parameters:
    chromosome (numpy array): Binary array of length 10 where 1 indicates a file is included, 0 indicates it is not.

    Returns:
    float: Fitness score based on weighted file values.
    """
    total_size = np.sum(chromosome * FILE_SIZES)
    if total_size <= CAPACITY:
        w = 1.0  # Feasible solution: full value
    else:
        w = 0.5  # Infeasible solution: halve the value of each selected file
    total_value = np.sum(chromosome * FILE_VALUES * w)
    return total_value

**Tournament Selection Function**

In [ ]:
# Tournament selection
def tournament_selection(population, fitnesses, tournament_size=3):
    indices = np.random.choice(len(population), tournament_size)
    best_idx = indices[np.argmax([fitnesses[i] for i in indices])]
    return population[best_idx]

**Single-point crossover function**

In [ ]:
# Single-point crossover
def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, len(parent1) - 1)
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
        return child1, child2
    return parent1.copy(), parent2.copy()

**Other Crossover Functions**

In [ ]:
def two_point_crossover(parent1, parent2):
    length = len(parent1)
    point1 = random.randint(0, length - 2)
    point2 = random.randint(point1 + 1, length - 1)

    offspring1 = np.concatenate([parent1[:point1], parent2[point1:point2], parent1[point2:]])
    offspring2 = np.concatenate([parent2[:point1], parent1[point1:point2], parent2[point2:]])

    return offspring1, offspring2


def uniform_crossover(parent1, parent2, crossover_prob=0.5):
    """
    Performs uniform crossover on two parent chromosomes.

    For each gene, a random decision is made whether to swap the genes
    from the parents, based on the crossover_prob.

    Args:
        parent1 (list): The first parent chromosome.
        parent2 (list): The second parent chromosome.
                        Must be the same length as parent1.
        crossover_prob (float): The probability of swapping a gene from
                                parent1 to parent2. Must be between 0.0 and 1.0.

    Returns:
        tuple: A tuple containing two new offspring chromosomes.
    """
    if len(parent1) != len(parent2):
        raise ValueError("Parents must have the same length for crossover.")

    length = len(parent1)
    offspring1 = [None] * length
    offspring2 = [None] * length

    for i in range(length):
        if random.random() < crossover_prob:
            offspring1[i] = parent2[i]
            offspring2[i] = parent1[i]
        else:
            offspring1[i] = parent1[i]
            offspring2[i] = parent2[i]

    return offspring1, offspring2

**Mutation**

In [ ]:
def mutate(chromosome):
    for i in range(len(chromosome)):
        if random.random() < MUTATION_RATE:
            chromosome[i] = 1 - chromosome[i]  # Flip bit
    return chromosome

**Initial Population**

In [ ]:
# Initialize population
def initialize_population(pop_size, num_files):
    return np.random.randint(2, size=(pop_size, num_files))

**Main GA function**

In [ ]:
def genetic_algorithm(id):
    start_time = time.time()
    population = initialize_population(POPULATION_SIZE, NUM_FILES)
    best_solution = None
    best_fitness = -float('inf')
    convergence_gen = None
    best_fitness_history = []
    total_best_fitness = 0

    for generation in range(GENERATIONS):
        fitnesses = [fitness(chrom) for chrom in population]
        max_fitness_idx = np.argmax(fitnesses)

        if fitnesses[max_fitness_idx] > best_fitness:
            best_fitness = fitnesses[max_fitness_idx]
            best_solution = population[max_fitness_idx].copy()
            convergence_gen = generation + 1

        best_fitness_history.append(best_fitness)

        new_population = []
        while len(new_population) < POPULATION_SIZE:
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.extend([child1, child2])

        population = np.array(new_population[:POPULATION_SIZE])
        population[0] = best_solution
        total_best_fitness += best_fitness

        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}, "
              f"Size = {np.sum(best_solution * FILE_SIZES)}")

    execution_time = time.time() - start_time
    mean_best_fitness = total_best_fitness / GENERATIONS

    # Plot convergence graph
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, GENERATIONS + 1), best_fitness_history, marker='o', linestyle='-', color='b')
    plt.title('GA Convergence: Best Fitness vs. Generation')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness (Total Value)')
    plt.grid(True)
    plt.savefig(f'convergence_plot_{id}.png')
    plt.close()

    return best_solution, best_fitness, convergence_gen, execution_time, mean_best_fitness

**Problem**

In [ ]:
# Problem parameters (10 files)
NUM_FILES = 10
FILE_VALUES = [121, 95, 85, 100, 78, 125, 130, 128, 135, 120]  # f(x): value of each file
FILE_SIZES = [800, 700, 650, 750, 600, 900, 950, 875, 1050, 1500]  # g(x): size of each file
CAPACITY = 4500  # Maximum DVD capacity

In [ ]:
CROSSOVER_RATE = 0.85
MUTATION_RATE = 0.1

In [ ]:
# Run GA and generate plot
configs = [
    {"id": 1, "pop_size": 50,  "generation": 20},
    {"id": 2, "pop_size": 50,  "generation":100},
    {"id": 3, "pop_size": 100, "generation":20},
    {"id": 4, "pop_size": 100, "generation":100}
]

for config in configs:
    print(f"\nTesting Config: {config}")

    POPULATION_SIZE = config["pop_size"]
    GENERATIONS = config["generation"]

    best_solution, best_fitness, convergence_gen, exec_time, mean_best_fitness = genetic_algorithm(config["id"])
    total_size = np.sum(best_solution * FILE_SIZES)
    print(f"Final Best Solution: {best_solution}")
    print(f"Final Best Fitness (Total Value): {best_fitness} minutes")
    print(f"Mean Best Fitness : {mean_best_fitness}")
    print(f"Total Size: {total_size} MB")
    print(f"Total Size as a percentage: {(total_size/CAPACITY * 100):.2f} %")
    print(f"Convergence Generation: {convergence_gen}")
    print(f"Execution Time: {exec_time:.2f} seconds")


Testing Config: {'id': 1, 'pop_size': 50, 'generation': 20}
Generation 1: Best Fitness = 596.0, Size = 4350
Generation 2: Best Fitness = 596.0, Size = 4350
Generation 3: Best Fitness = 599.0, Size = 4325
Generation 4: Best Fitness = 606.0, Size = 4400
Generation 5: Best Fitness = 609.0, Size = 4375
Generation 6: Best Fitness = 611.0, Size = 4450
Generation 7: Best Fitness = 613.0, Size = 4475
Generation 8: Best Fitness = 613.0, Size = 4475
Generation 9: Best Fitness = 613.0, Size = 4475
Generation 10: Best Fitness = 613.0, Size = 4475
Generation 11: Best Fitness = 613.0, Size = 4475
Generation 12: Best Fitness = 613.0, Size = 4475
Generation 13: Best Fitness = 613.0, Size = 4475
Generation 14: Best Fitness = 613.0, Size = 4475
Generation 15: Best Fitness = 613.0, Size = 4475
Generation 16: Best Fitness = 613.0, Size = 4475
Generation 17: Best Fitness = 613.0, Size = 4475
Generation 18: Best Fitness = 613.0, Size = 4475
Generation 19: Best Fitness = 613.0, Size = 4475
Generation 20: Be

**Genetic Algorithm With Two point Crossover**

In [ ]:
def genetic_algorithm_with_tpc(id):
    start_time = time.time()
    population = initialize_population(POPULATION_SIZE, NUM_FILES)
    best_solution = None
    best_fitness = -float('inf')
    convergence_gen = None
    best_fitness_history = []
    total_best_fitness = 0

    for generation in range(GENERATIONS):
        fitnesses = [fitness(chrom) for chrom in population]
        max_fitness_idx = np.argmax(fitnesses)

        if fitnesses[max_fitness_idx] > best_fitness:
            best_fitness = fitnesses[max_fitness_idx]
            best_solution = population[max_fitness_idx].copy()
            convergence_gen = generation + 1

        best_fitness_history.append(best_fitness)

        new_population = []
        while len(new_population) < POPULATION_SIZE:
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child1, child2 = two_point_crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.extend([child1, child2])

        population = np.array(new_population[:POPULATION_SIZE])
        population[0] = best_solution
        total_best_fitness += best_fitness

        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}, "
              f"Size = {np.sum(best_solution * FILE_SIZES)}")

    execution_time = time.time() - start_time
    mean_best_fitness = total_best_fitness / GENERATIONS

    # Plot convergence graph
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, GENERATIONS + 1), best_fitness_history, marker='o', linestyle='-', color='b')
    plt.title('GA Convergence: Best Fitness vs. Generation')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness (Total Value)')
    plt.grid(True)
    plt.savefig(f'convergence_plot_with_tpc_{id}.png')
    plt.close()

    return best_solution, best_fitness, convergence_gen, execution_time, mean_best_fitness

In [ ]:
# Run GA and generate plot
configs = [
    {"id": 1, "pop_size": 50,  "generation": 20},
    {"id": 2, "pop_size": 50,  "generation":100},
    {"id": 3, "pop_size": 100, "generation":20},
    {"id": 4, "pop_size": 100, "generation":100}
]

for config in configs:
    print(f"\nTesting Config: {config}")

    POPULATION_SIZE = config["pop_size"]
    GENERATIONS = config["generation"]

    best_solution, best_fitness, convergence_gen, exec_time, mean_best_fitness = genetic_algorithm_with_tpc(config["id"])
    total_size = np.sum(best_solution * FILE_SIZES)
    print(f"Final Best Solution: {best_solution}")
    print(f"Final Best Fitness (Total Value): {best_fitness} minutes")
    print(f"Mean Best Fitness : {mean_best_fitness}")
    print(f"Total Size: {total_size} MB")
    print(f"Total Size as a percentage: {(total_size/CAPACITY * 100):.2f} %")
    print(f"Convergence Generation: {convergence_gen}")
    print(f"Execution Time: {exec_time:.2f} seconds")


Testing Config: {'id': 1, 'pop_size': 50, 'generation': 20}
Generation 1: Best Fitness = 609.0, Size = 4375
Generation 2: Best Fitness = 609.0, Size = 4375
Generation 3: Best Fitness = 611.0, Size = 4450
Generation 4: Best Fitness = 611.0, Size = 4450
Generation 5: Best Fitness = 611.0, Size = 4450
Generation 6: Best Fitness = 611.0, Size = 4450
Generation 7: Best Fitness = 611.0, Size = 4450
Generation 8: Best Fitness = 614.0, Size = 4425
Generation 9: Best Fitness = 614.0, Size = 4425
Generation 10: Best Fitness = 614.0, Size = 4425
Generation 11: Best Fitness = 614.0, Size = 4425
Generation 12: Best Fitness = 614.0, Size = 4425
Generation 13: Best Fitness = 614.0, Size = 4425
Generation 14: Best Fitness = 614.0, Size = 4425
Generation 15: Best Fitness = 614.0, Size = 4425
Generation 16: Best Fitness = 614.0, Size = 4425
Generation 17: Best Fitness = 614.0, Size = 4425
Generation 18: Best Fitness = 614.0, Size = 4425
Generation 19: Best Fitness = 614.0, Size = 4425
Generation 20: Be